# Generic HMC 

### Imports

In [ ]:
import os
import sys
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from l2hmc_eager import dynamics_eager as _l2hmc
from l2hmc_eager import gauge_dynamics_eager as l2hmc
from l2hmc_eager.neural_nets import *
from utils.distributions import GMM, gen_ring
from utils.jacobian import _map, jacobian
from utils.data_utils import (
    calc_avg_vals_errors, block_resampling, jackknife_err
)

from HMC.hmc import HMC

from lattice.gauge_lattice import GaugeLattice, pbc, mat_adj, u1_plaq_exact

%autoreload 2

In [ ]:
from u1_model_eager import *

In [ ]:
tf.enable_eager_execution()
tfe = tf.contrib.eager

In [ ]:
from gauge_model import GaugeModel

import utils.gauge_model_helpers as helpers

## Helper functions

In [ ]:
def compute_ac_spectrum(samples_history, target_mean, target_covar):
    """Compute autocorrelation spectrum.
    Follows equation 15 from the L2HMC paper.
    Args:
        samples_history: Numpy array of shape [T, B, D], where T is the total
            number of time steps, B is the batch size, and D is the dimensionality
            of sample space.
        target_mean: 1D Numpy array of the mean of target(true) distribution.
        target_covar: 2D Numpy array representing a symmetric matrix for 
            variance.
    Returns:
        Autocorrelation spectrum, Numpy array of shape [T-1].
    """
    # Using numpy here since eager is a bit slow due to the loop
    time_steps = samples_history.shape[0]
    #trace = np.trace(target_covar)
    trace = 1.
    rhos = []
    for t in range(time_steps - 1):
        rho_t = 0.
        for tau in range(time_steps - t):
            v_tau = samples_history[tau, :] - target_mean
            v_tau_plus_t = samples_history[tau + t, :] - target_mean
            # Take dot product over observation dims and take mean over batch dims
            rho_t = v_tau.T.dot(v_tau_plus_t)
            #rho_t += np.mean(np.sum(v_tau * v_tau_plus_t, axis=1))
        rho_t /= trace * (time_steps - t)
        rhos.append(rho_t)
    return np.array(rhos)

## 2D $U(1)$ Lattice Gauge Theory

### Using L2HMC framework with hmc flag. `hmc=True`

In [ ]:
params = {
    'time_size': 8,
    'space_size': 8,
    'link_type': 'U1',
    'dim': 2,
    'beta': 8.,
    'num_samples': 5,
    'num_steps': 10, 
    'eps': 0.2,
    'loss_scale': 0.1,
    'loss_eps': 1e-4,
    'learning_rate_init': 1e-4,
    'learning_rate_decay_steps': 100,
    'learning_rate_decay_rate': 0.96,
    'train_steps': 5000,
    #'record_loss_every': 50,
    #'data_steps': 1,
    'save_steps': 500,
    #'print_steps': 1,
    'logging_steps': 25,
    'clip_value': 100,
    'rand': False,
    'metric': 'l2',
    #'conv_net': False,
    #'hmc': True,
}
tf.reset_default_graph()

In [ ]:
del model_hmc

In [ ]:
tf.reset_default_graph()
config=tf.ConfigProto()

In [ ]:
model_hmc = GaugeModel(params=params,
                       config=config,
                       sess=None,
                       conv_net=False,
                       hmc=True,
                       log_dir=None,
                       restore=False,
                       eps_trainable=False,
                       aux=False)

In [ ]:
model_hmc.train(1000, kill_sess=False)

In [ ]:
samples_history = model_hmc.run(250)

In [ ]:
#model_hmc = GaugeModelEager(params=params,
                            #conv_net=False,
                            #hmc=True,
                            #log_dir=None,
                            #restore=False,
                            #defun=False,
                            #eps_trainable=False)

In [ ]:
observables_hmc = model_hmc.calc_observables(model_hmc.dynamics.samples, 
                                             update=True)
total_actions, avg_plaquettes, top_charges = observables_hmc
helpers.print_run_data(model_hmc.data, header=True)
helpers.write_run_data(model_hmc.files['run_info_file'],  model_hmc.data)

In [ ]:
_, _, _, samples_out = model_hmc.dynamics.apply_transition(model_hmc.samples)

In [ ]:
model_hmc.train(500)

In [ ]:
steps_arr = [0]
steps_arr.extend(model_hmc.steps_arr)

_ = helpers.plot_run_data(model_hmc.data, model_hmc.params, steps_arr, 
                          model_hmc.figs_dir, skip_steps=1)

In [ ]:
samples_hmc = tf.random_normal(shape=model_hmc.samples.shape)
samples_history_hmc = []
actions_history_hmc = []
avg_plaquettes_history_hmc = []
top_charges_history_hmc = []

In [ ]:
for i in range(500):
    #samples_history_hmc.append(samples_hmc.numpy())
    t0 = time.time()
    #_, _, _, samples_hmc = apply_transition_hmc(samples_hmc)
    _, _, _, samples_hmc = model_hmc.dynamics.apply_transition(samples_hmc)
    observables_hmc = np.array(
        model_hmc.lattice.calc_plaq_observables(samples_hmc)
    ).T
    actions_history_hmc.append(observables_hmc[0])
    avg_plaquettes_history_hmc.append(observables_hmc[1])
    top_charges_history_hmc.append(observables_hmc[2])
    step_time = (time.time() - t0) / (model_hmc.num_steps * model_hmc.batch_size)
    print(f'step: {i}  time/step: {step_time:^6.4g} '
          f'top_charge: {np.mean(observables_hmc[2]):^6.4g} '
          f'avg_plaq: {np.mean(observables_hmc[1]):^6.4g}')

In [ ]:
actions_history_hmc = np.array(actions_history_hmc)
avg_plaquettes_history_hmc = np.array(avg_plaquettes_history_hmc)
top_charges_history_hmc = np.array(top_charges_history_hmc)

In [ ]:
top_charges_autocorr0_hmc = autocorr(top_charges_history_hmc[:, 0])
top_charges_autocorr1_hmc = autocorr(top_charges_history_hmc[:, 1])

top_charges_autocorr_hmc = (top_charges_autocorr0_hmc 
                            + top_charges_autocorr1_hmc) / 2

In [ ]:
steps_hmc = np.arange(len(top_charges_autocorr_hmc))

fig, ax = plt.subplots()
#ax.plot(steps_hmc, top_charges_autocorr_hmc, 
#        marker='', ls='-', label='topological_charge (HMC)')
#ax.plot(steps_hmc, avg_plaquettes_autocorr_hmc,
#        marker='', ls='--', label='avg plaquettes (L2HMC, hmcNet)')
ax.plot(steps_hmc, top_charges_autocorr0_hmc,
        marker='', ls='-', label='topological charges (GenericHMC, sample1)')
ax.plot(steps_hmc, top_charges_autocorr1_hmc,
        marker='', ls='--', label='topological charges (GenericHMC, sample2)')
ax.plot(steps_hmc, top_charges_autocorr_hmc,
        marker='', ls=':', label='topological charges (GenericHMC, mean)')
#ax.plot(steps1, top_charges_autocorr, marker='', ls='-', label='topological_charge (L2HMC)')
ax.set_title(r"$\epsilon = 0.1$, 10 steps", fontsize=16)
ax.set_ylabel('Autocorrelation', fontsize=14)
ax.set_xlabel('Gradient Evaluations', fontsize=14)
ax.legend(loc='best', fontsize=12)
fig.savefig(os.path.join(model_hmc.figs_dir, 
                         'top_charge_autocorrelation_fn_hmc.pdf'), 
            dpi=400, bbox_inches='tight')
#ax.set_xlim((ax.get_xlim()[0], 1000))
#ax.set_xlim((-5, 1000))
plt.show()

### Using HMC.hmc method (separate from L2HMC)

In [ ]:
dirs = helpers.create_log_dir('gauge_logs_graph/HMC')
log_dir, info_dir, figs_dir = dirs

In [ ]:
params = {
    'time_size': 16,
    'space_size': 16,
    'link_type': 'U1',
    'dim': 2,
    'beta': 8.,
    'num_samples': 5,
    'num_steps': 5, 
    'eps': 0.1,
    'loss_scale': 0.1,
    'loss_eps': 1e-4,
    'learning_rate_init': 1e-4,
    'learning_rate_decay_steps': 100,
    'learning_rate_decay_rate': 0.96,
    'train_steps': 1000,
    'record_loss_every': 50,
    'data_steps': 1,
    'save_steps': 50,
    'print_steps': 1,
    'logging_steps': 5,
    'clip_value': 100,
    'rand': False,
    'metric': 'l2',
    #'conv_net': False,
    #'hmc': True,
}

In [ ]:
lattice = GaugeLattice(time_size=params['time_size'],
                       space_size=params['space_size'],
                       dim=params['dim'],
                       beta=params['beta'],
                       link_type=params['link_type'],
                       num_samples=params['num_samples'],
                       rand=params['rand'])
samples = np.array([sample.flatten() for sample in lattice.samples])
lattice_energy_fn = lattice.get_energy_function(samples)

In [ ]:
#position_init = lattice.links.flatten()
lattice_hmc = HMC(position_init=samples,
                  step_size=params['eps'],
                  n_leapfrog_steps=params['num_steps'],
                  potential_fn=lattice_energy_fn,
                  grad_potential_fn=None,
                  beta=lattice.beta)
print("Exact value of the average plaquette "
      f"(at {params['beta']}): {u1_plaq_exact(params['beta'])}")

#### Run HMC algorithm

In [ ]:
momentum = np.random.randn(*samples.shape)

samples_history_dir = os.path.join(log_dir, 'samples_history')
if not os.path.isdir(samples_history_dir):
    os.makedirs(samples_history_dir)

eval_steps_arr = [50, 100, 200, 400, 500, 600, 800, 1000, 5000]
for eval_steps in eval_steps_arr:
    samples_arr = []
    probs_arr = []
    for i in range(eval_steps):
        t1 = time.time()
        if isinstance(samples, tf.Tensor):
            samples_arr.append(samples.numpy())
        else: 
            samples_arr.append(samples)
        samples, vel, probs = lattice_hmc.apply_transition(samples)
        probs_arr.append(probs)
        
        tt = (time.time() - t1)
              #/ (params['num_steps'] 
              #   * lattice.num_samples 
              #   * lattice.num_links))
        print(f"\nstep: {i:<5g} accept rate: {np.mean(probs):^8.5g}  "
              f" time/step: {tt:^6.4g} ")
        samples_history_file = os.path.join(
            samples_history_dir,
            f'samples_history_{eval_steps}.pkl'
        )
        accept_prob_history_file = os.path.join(
            samples_history_dir,
            f'accept_prob_history_{eval_steps}.pkl'
        )
        with open(samples_history_file, 'wb') as f:
            pickle.dump(samples_arr, f)
        with open(accept_prob_history_file, 'wb') as f:
            pickle.dump(probs_arr, f)

In [ ]:
samples_arr = np.array(samples_arr)
samples_arr.shape

In [ ]:
samples_arr[0][0][:10]
samples_arr[1][0][:10]

In [ ]:
import pickle
samples_history_file = os.path.join(info_dir, 'samples_history.pkl')
parameters_file = os.path.join(info_dir, 'parameters.pkl')
with open(samples_history_file, 'wb') as f:
    pickle.dump(np.array(samples_arr), f)
with open(parameters_file, 'wb') as f:
    pickle.dump(params, f)

In [ ]:
observables = np.array([
    lattice.calc_plaq_observables(samples=samples_arr[i]) for i in range(100)
])
#total_actions, avg_plaquettes, top_charges = observables

In [ ]:
observables.shape

In [ ]:
actions = observables[:, 0, :]
plaquettes = observables[:, 1, :]
charges = observables[:, 2, :]

In [ ]:
actions.shape

In [ ]:
COLORS = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
MARKERS = ['o', 's', 'x', 'v', 'h', '^', 'p', '<', 'd', '>', 'o']
LINESTYLES = ['-', '--', ':', '-.', '-', '--', ':', '-.', '-', '--']

In [ ]:
fig, ax = plt.subplots()
for idx in range(actions.shape[1]):
    _ = ax.plot(actions[:, idx], label=f'Sample {idx}', color=COLORS[idx])
    
_ = ax.legend(loc='best')
_ = ax.set_xlabel('Step')
_ = ax.set_ylabel('Total action')

In [ ]:
fig, ax = plt.subplots()
for idx in range(plaquettes.shape[1]):
    _ = ax.plot(plaquettes[:, idx], label=f'Sample {idx}', color=COLORS[idx], ls='--')

_ = ax.plot(plaquettes.mean(axis=1), label="Average", color='k', alpha=0.8,
            lw=3.)
    
_ = ax.axhline(y=u1_plaq_exact(lattice.beta), 
               color='r', ls='-', lw=2.5, label='exact')
    
_ = ax.legend(loc='best')
_ = ax.set_xlabel('Step')
_ = ax.set_ylabel('Average Plaquette')

In [ ]:
fig, ax = plt.subplots()
for idx in range(charges.shape[1]):
    _ = ax.plot(charges[:, idx], label=f'Sample {idx}', color=COLORS[idx])
    
_ = ax.legend(loc='best')
_ = ax.set_xlabel('Step')
_ = ax.set_ylabel('Topological charge')

#### Compute autocorrelations

In [ ]:
total_actions = np.array(total_actions)
avg_plaquettes = np.array(avg_plaquettes)
top_charges = np.array(top_charges)
samples_arr = np.array(samples_arr)
print(total_actions.shape, avg_plaquettes.shape, top_charges.shape)

In [ ]:
top_charges_autocorr = autocorr(top_charges)

links_autocorr_arr = [
    autocorr(sample) for sample in samples_arr.T
]

links_autocorr_arr = np.array(links_autocorr_arr)

#### Save samples, params

In [ ]:
samples_history_file = os.path.join(info_dir, 'sample_history.pkl')
with open(samples_history_file, 'wb') as f:
    _ = pickle.dump(samples_arr, f)
    
params_file = os.path.join(info_dir, 'parameters.pkl')
with open(params_file, 'wb') as f:
    _ = pickle.dump(samples_arr, f)
    
params_txt_file = os.path.join(info_dir, 'parameters.txt')
with open(params_txt_file, 'w') as f:
    for key, val in params.items():
        _ = f.write(f'{key}: {val}\n')

#### Plot autocorrelation of top. charge and individual links

In [ ]:
%matplotlib notebook

In [ ]:
steps_hmc = np.arange(len(top_charges_autocorr))

fig, ax = plt.subplots()
ax.plot(steps_hmc, top_charges_autocorr,
        marker='', ls='-')#, label='')
title_str = (rf"$\epsilon =$ {params['eps']}, {params['num_steps']} steps")
ax.set_title(title_str, fontsize=16)
ax.set_ylabel('Autocorrelation (top. charge)', fontsize=14)
ax.set_xlabel('step', fontsize=14)
ax.legend(loc='best', fontsize=12)
out_file = os.path.join(
    figs_dir,  'top_charge_autocorrelation_fn_hmc.pdf'
)
plt.savefig(out_file, dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
steps = np.arange(len(links_autocorr_arr.T))
fig, ax = plt.subplots()
for i in range(10):
    _ = ax.plot(steps, links_autocorr_arr[i, :], label=f'link {i}', ls='-',
                alpha=0.7)
    #_ = ax.plot(acl_steps, samples_acl_spectrum/samples_acl_spectrum[0], 
    #            label=f'{key}')
_ = ax.plot(steps, links_autocorr_arr.mean(axis=0), label='average',
            color='k', lw=2.5)
_ = ax.set_xlabel('step', fontsize=14)
_ = ax.set_ylabel('Autocorrelation (individual links)', fontsize=14)
#_ = ax.legend(loc='best')
plt.savefig(os.path.join(figs_dir, 'links_autocorrelation_vs_step.pdf'),
            dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
steps = np.arange(len(links_autocorr_arr.T))
fig, ax = plot_multiple_lines(steps, links_autocorr_arr[:10, :], 
                              x_label='steps', 
                              y_label='Autocorrelation (links)')

# OLD

In [ ]:
target_mean = np.mean(samples_arr, axis=0)

In [ ]:
samples_autocorr = compute_ac_spectrum(samples_arr, target_mean=target_mean,
                                       target_covar=None)

In [ ]:
steps_hmc = np.arange(len(samples_autocorr))

fig, ax = plt.subplots()
ax.plot(steps_hmc, samples_autocorr,
        marker='', ls='-', label='samples (hmc.HMC)')
ax.set_title(r"$\epsilon = 0.05$, 10 steps", fontsize=16)
ax.set_ylabel('Autocorrelation', fontsize=14)
ax.set_xlabel('Gradient Evaluations', fontsize=14)
ax.legend(loc='best', fontsize=12)
#fig.savefig(os.path.join(model_hmc.figs_dir, 
#                         'top_charge_autocorrelation_fn_hmc.pdf'), 
#            dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
apply_transition = tfe.defun(lattice_hmc.apply_transition)

In [ ]:
_samples = tf.random_normal(shape=links1.shape)
#_samples = np.random.randn(*links1.shape)
samples_arr = []
actions_arr = []
plaquettes_arr = []
top_charges_arr = []
for i in range(500):
    samples_arr.append(_samples)
    _samples, _, _ = apply_transition(_samples)
    
    observables = np.array(lattice._calc_plaq_observables(_samples))
    _total_actions = observables[0]
    _avg_plaquettes = observables[1]
    _top_charges = observables[2]
    
    actions_arr.append(_total_actions)
    plaquettes_arr.append(_avg_plaquettes)
    top_charges_arr.append(_top_charges)

In [ ]:

#apply_transition = tfe.defun(lattice_hmc.apply_transition)

#_samples = tf.random_normal(shape=links1.shape)
#_samples = np.random.randn(*links1.shape)
#samples_arr = []
#actions_arr = []
#plaquettes_arr = []
#top_charges_arr = []
#for i in range(100):
    #samples_arr.append(_samples)
    #_samples, _, _ = apply_transition(_samples)
    #
    #observables = np.array(lattice._calc_plaq_observables(_samples))
    #_total_actions = observables[0]
    #_avg_plaquettes = observables[1]
    #_top_charges = observables[2]
    #
    #actions_arr.append(_total_actions)
    #plaquettes_arr.append(_avg_plaquettes)
    #top_charges_arr.append(_top_charges)

## Create plots (old)

In [ ]:
%matplotlib notebook

In [ ]:
steps = np.arange(len(top_charge_autocorr))
fig, ax = plt.subplots()
ax.plot(steps, top_charge_autocorr, ls='-', marker='', 
        label=f"HMC (eps: {params['eps']}, 5 steps)")
ax.plot(steps, top_charge_autocorr1, ls='-', marker='', 
        label=f"HMC (eps: {0.1}, 5 steps)")
ax.plot(steps, top_charge_autocorr2, ls='-', marker='', 
        label=f"HMC (eps: {0.025}, 5 steps)")
ax.plot(steps, top_charge_autocorr, ls='-', marker='', 
        label=f"HMC (eps: {0.05}, 10 steps)")
ax.plot(steps, top_charge_autocorr4, ls='-', marker='', 
        label=f"HMC (eps: {0.025}, 10 steps)")
ax.set_xlabel('Gradient computations')
ax.set_ylabel('Autocorrelation of Topological Charge')
#ax.set_xlim((-2, 50))
ax.legend(loc='best')
fig.savefig(
    '../../figures/HMC_autocorrelation_fn/top_charge_autocorr_no_l2hmc.pdf', 
    dpi=400, bbox_inches='tight'
)
plt.show()

In [ ]:
steps = np.arange(len(samples_autocorr))
fig, ax = plt.subplots()
ax.semilogy(steps, samples_autocorr, ls='-', marker='', 
        label=f"HMC (eps: {params['eps']}, 5 steps)")
ax.set_xlabel('Gradient computations')
ax.set_ylabel('Autocorrelation from Samples')
#ax.set_xlim((-2, 50))
ax.legend(loc='best')
#fig.savefig(
#    '../../figures/HMC_autocorrelation_fn/top_charge_autocorr_no_l2hmc.pdf', 
#    dpi=400, bbox_inches='tight'
#)
plt.show()

In [ ]:
np.mean(average_plaquettes)

In [ ]:
print(u1_plaq_exact(beta))

## Using L2HMC with auxiliary functions $Q, S, T \equiv 0$ (i.e. generic HMC)

In [ ]:
##########################  Parameters  #####################################
# n_steps: number of leapfrog steps, eps: initial step size for dynamics
# loss_scale: scaling factor (lambda^2 in paper) in loss objective
# loss_eps: for numeric stability in loss function
# beta: inverse coupling strength
##############################################################################
time_size, space_size, dim, beta, num_samples = (4, 4, 2, 3., 4)
n_steps, eps, loss_scale, loss_eps = (10, 0.1, .1, 1e-4)
rand=True
l2_dist = True
conv_net = True

In [ ]:
u1_lattice = GaugeLattice(time_size, space_size, dim, beta,
                          link_type='U1', num_samples=num_samples, rand=rand)
if conv_net:
    u1_samples_tensor = tf.convert_to_tensor(u1_lattice.samples, 
                                             dtype=tf.float32)
else:
    flat_samples = [sample.flatten() for sample in u1_lattice.samples]
    u1_samples_tensor = tf.convert_to_tensor(np.stack(flat_samples), 
                                             dtype=tf.float32)

# Construct dynamics object
u1_energy_fn = u1_lattice.get_energy_function(u1_samples_tensor)
u1_dynamics = l2hmc.GaugeDynamics(u1_lattice, n_steps=n_steps, eps=eps,
                                  minus_loglikelihood_fn=u1_energy_fn, 
                                  conv_net=conv_net, test_HMC=True)

In [ ]:
global_step = tf.train.get_or_create_global_step()
_ = global_step.assign(1)
train_iters = 500
record_loss_every = 50
save_steps = 50 

learning_rate = tf.train.exponential_decay(1e-2, global_step, 50,
                                           0.96, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate)
checkpointer = tf.train.Checkpoint(
    optimizer=optimizer, dynamics=u1_dynamics, global_step=global_step
)
#summary_writer = tf.contrib.summary.create_file_writer(log_dir)
loss_fn = l2hmc.compute_loss

print(u1_plaq_exact(beta))

In [ ]:
#################    Run L2HMC algorithm    ##################################
total_actions = []
average_plaquettes = []
topological_charges = []
samples = u1_samples_tensor

In [ ]:
t0 = time.time()
start_step = global_step.numpy()
for i in range(start_step, 1000):
    t1 = time.time()
    loss, samples, accept_prob = train_one_iter(
        u1_dynamics,
        samples,
        optimizer,
        loss_fn=loss_fn,
        scale=loss_scale,
        eps=loss_eps,
        global_step=global_step
    )
    observables = np.array(u1_lattice.calc_plaq_observables(samples))
    _total_actions = observables[:, 0]
    _avg_plaquettes = observables[:, 1]
    _top_charges = observables[:, 2]
    
    total_actions.append(_total_actions)
    average_plaquettes.append(_avg_plaquettes)
    topological_charges.append(_top_charges)
    
    print(f'\nstep: {i:<5g} loss: {loss.numpy():^8.5g} '
          f' time/step: {time.time() - t1:^6.4g} '
          f' accept: {accept_prob.numpy().mean():^8.5g} '
          f' eps: {u1_dynamics.eps.numpy():^6.4g} '
          f' avg_S: {np.mean(_total_actions):^8.5g} '
          f' avg_topQ: {np.mean(_top_charges):^8.5g} '
          f' avg_plaq: {np.mean(_avg_plaquettes):^8.5g}\n ')
    print('avg_plaquettes: {}\n'.format([_avg_plaquettes]))

In [ ]:
samples = u1_samples_tensor
print(samples.shape)

In [ ]:
x = tf.reshape(samples, shape=[samples.shape[0], -1])
y = tf.random_normal(x.shape)

In [ ]:
xy = tf.matmul(x, y, transpose_b=True)

In [ ]:
xy_loss = tf.reduce_sum(xy / (tf.norm(x) * tf.norm(y)), axis=1)

In [ ]:
loss = tf.reduce_mean((loss_scale / xy_loss - xy_loss / loss_scale), axis=0)

In [ ]:
loss

In [ ]:
help(tf.clip_by_global_norm)

In [ ]:
tf.abs

In [ ]:
ss = tf.matmul(samples, samples)
print(ss.shape)

In [ ]:
avg_plaqs_arr = np.array(average_plaquettes)
_avg_plaqs_arr = np.mean(avg_plaqs_arr, axis=0)
avg_plaq, avg_plaq_err = calc_avg_vals_errors(avg_plaqs_arr[450:500], num_blocks=50)
print(f'avg_plaq (mean from arr): {np.mean(_avg_plaqs_arr)}')
print(f'avg_plaq: {avg_plaq} +/- {avg_plaq_err}')

In [ ]:
np.mean(average_plaquettes[-100:])

In [ ]:
def project_angle(x):
    """Function to project an angle from [-4pi, 4pi] to [-pi, pi]."""
    return x - 2 * np.pi * tf.math.floor((x + np.pi) / (2 * np.pi))

In [ ]:
project_angle(-2 * np.pi)

In [ ]:
t = np.arange(-10, 10, 0.05)
y = project_angle(t)

In [ ]:
fig, ax = plt.subplots()
ax.plot(t, y, 'o')

## Strongly Correlated Gaussian target distribution (for testing HMC implementation)

### Define log density function of target distribution (potential energy function) $S(x)$

$$ S(\mathbf{x}) = \frac{-\frac{1}{2} (\mathbf{x} - \mathbf{\mu})^{T} \mathbf{\Sigma}^{-1}(\mathbf{x} - \mathbf{\mu})}{\sqrt{|\Sigma|}} $$ 


In [ ]:
mu = np.zeros(2)
cov = np.array([[1., 0.95], [0.95, 1.]], dtype=np.float32)
cov_inv = np.linalg.inv(np.copy(cov))
def quadratic_gaussian(x):
    x_mu = x - mu
    return 0.5 * tf.reduce_sum(tf.transpose(x_mu) * cov_inv * x_mu)

def quadratic_gaussian_grad(x):
    return x

### Exact target distribution

In [ ]:
mean = [0, 0]
cov = [[1., 0.95], [0.95, 1.]]
samples_x, samples_y = np.random.multivariate_normal(mean, cov, 1000).T

### Instantiate HMC object for sampling

In [ ]:
step_size = 0.1
n_leapfrog_steps = 15
hmc = HMC(position_init=np.random.randn(2), 
          step_size=step_size,
          n_leapfrog_steps=n_leapfrog_steps,
          potential_fn=quadratic_gaussian,
          grad_potential_fn=grad_quad_gaussian)
          #grad_potential_fn=grad_quad_gaussian)

In [ ]:
pos0 = [[-1., 1.]]
pos = [pos0]
vel = []
probs = []
pos1 = pos0

for i in range(500):
    #pos0 = pos[i-1]
    pos1, vel1, probs1 = hmc.apply_transition(pos1)
    pos.append(pos1)
    vel.append(vel1)
    probs.append(probs1)
pos = np.array(pos).reshape(len(pos), -1)
vel = np.array(vel)
probs = np.array(probs)

In [ ]:
fig, ax = plt.subplots()
_ = ax.plot(samples_x, samples_y, marker='o', ls='', alpha=0.45)
_ = ax.plot(pos[:,0], pos[:,1], marker='o', ls='', alpha=0.6)
_ = ax.set_title('500 transitions, numerical gradient')
plt.show()

### Look at the leapfrog integrator to tune hyperparameters

In [ ]:
x0 = np.array([-1., 1.])
p0 = np.random.randn(*np.array(x0).shape)
x, p = x0, p0
x_arr = []
p_arr = []
for i in range(n_leapfrog_steps):
    lf_out = hmc._leapfrog_fn(x, p, i)
    x, p = lf_out
    x_arr.append(x)
    p_arr.append(p)
x_arr = np.array(x_arr)
p_arr = np.array(p_arr)

In [ ]:
fig, ax = plt.subplots()
_ = ax.plot(samples_x, samples_y, marker='o', ls='', alpha=0.45)
_ = ax.plot(x_arr[:,0], x_arr[:,1], marker='.', ls='-', alpha=0.6)
plt.show()

In [ ]:
fig, ax = plt.subplots()
_ = ax.plot(samples_x, samples_y, marker='o', ls='', alpha=0.45)
_ = ax.plot(pos[:,0], pos[:,1], marker='o', ls='', alpha=0.6)
_ = ax.set_title('500 transitions, numerical gradient')
plt.show()